In [ ]:
## Import Library
import keras
import tensorflow as tf
from keras import optimizers
from keras import regularizers
import os
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense
import pandas as pd
import numpy as np
import random as rn
np.random.seed(42)
tf.set_random_seed(1234)

In [ ]:
## Read Data
train=pd.read_csv('/Dataset/train_data_one_to_one.csv')
test=pd.read_csv('/Dataset/test_data.csv')

## Label defining
y_train=train.loc[:,'101']
y_test=test.loc[:,'101']
y_train=y_train.append(y_test,ignore_index=True)
y_train=pd.get_dummies(y_train)     # to one hot_encoding
y_test=y_train[580:]
y_train=y_train[0:580]


## Features Extraction
x_train=train.drop(train.columns[[0,101]],axis=1,inplace=False)
x_test=test.drop(test.columns[[0,101]],axis=1,inplace=False)

## Convert to numpy
x_train=np.array(x_train)
x_test=np.array(x_test)
y_train=np.array(y_train)
y_test=np.array(y_test)

## To select any row
#print(data.iloc[0:2])
## To selct any column
#print(data.loc[:,'0'])


In [ ]:
## To Obtain Reproducible result
os.environ['PYTHONHASHSEED'] = '0'
np.random.seed(42)
rn.seed(12345)
session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
tf.set_random_seed(1234)
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

## Model Creation
model=Sequential()
model.add(Dense(64,activation ='relu',bias_initializer='glorot_normal',
                kernel_regularizer=regularizers.l2(0.005),bias_regularizer=regularizers.l2(0.1),input_dim=100,name='FC_1'))
model.add(Dense(32,activation='relu',bias_initializer='glorot_normal',
                kernel_regularizer=regularizers.l2(0.005),bias_regularizer=regularizers.l2(0.1),name='FC_2'))
model.add(Dense(32,activation='relu',bias_initializer='glorot_normal',
                 kernel_regularizer=regularizers.l2(0.005),bias_regularizer=regularizers.l2(0.1),name='FC_3'))
model.add(Dense(6,activation='softmax',name='softmax_layer'))

adam=optimizers.Adam(lr=0.00001)

model.compile(loss='categorical_crossentropy',optimizer=adam,metrics=['accuracy'])
model.fit(x_train,y_train,epochs=3900,batch_size=16,validation_data=[x_test,y_test])
model.evaluate(x_test,y_test,batch_size=1)


In [ ]:
## Evaluating model
actual_result=test.loc[:,'101']                      ## Desired_output
pred=model.predict_classes(x_test)                   ## Predicted_result
class_names=['Anemometer errors','BladeAccumulatorPressureIssues', 'CoolingSystemIssues', 'Generator speed discrepancies', 'Oil Leakage', 'Overheated oil']
pred_result=[]
j=0
## To convert integer to class_names
for i in pred:
    pred_result=np.append(pred_result,class_names[i])
    ## Printing Mislaballed data
    if(actual_result[j] != pred_result[j]):
        print("MisLabelled: Index:",j,",Actual_result:",actual_result[j],", Predicted_result:",pred_result[j])
    #else:
    #    print("Correct: Index:",j,",Actual_result:",actual_result[j],", Predicted_result:",pred_result[j])
    j=j+1

In [ ]:
## Demo of model
import matplotlib.pyplot as plt
%matplotlib inline

## Change the index the sample will change - use any value between 0 and 59
index=0

## Loading a sample Input and its actual result
demo_y=test.loc[index,'101']
demo_x=x_test[index]

## Plot the sample
plt.plot(demo_x)
plt.show()
demo_x=demo_x.reshape([1,100])
print("Actual:",demo_y)

## Predict the output from model
y_pred=model.predict_classes(demo_x)
print(y_pred)
print("Prediction:",class_names[y_pred[0]])